In [1]:
%matplotlib inline
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
# dimension of input
dims = 80

signal_length = 100
signal_repeats = 3
predict_ahead = 1
noise_strength = 0.03
total_series_length = signal_length * signal_repeats

In [3]:
def generateData(dims, signal_length, predict_ahead, 
                 signal_repeats, batch_size, noise_strength):
    total_series_length = signal_length * signal_repeats
    time = np.linspace(
        0, np.pi*2*signal_repeats, 
        total_series_length * dims + predict_ahead * dims, dtype=np.float32
    )
    time = time.reshape((1, -1, dims))  
    
    # include shift for batches
    time = np.repeat(time, batch_size, 0)
    time += np.random.random(batch_size)[:, None, None] * 10
    y = np.sin(time)
    input_ = y[:, :total_series_length, :].copy()
    if noise_strength > 0:
        input_ += np.random.normal(size=(input_.shape)) * noise_strength # add some noise to the input
    target = y[:, predict_ahead:, :]
    return torch.tensor(input_) , torch.tensor(target)

In [4]:
# first, define our model and initialize the learnable weights and biases (parameters):
class LSTM(nn.Module):
    def __init__(self, dims, hidden_dims,
                 learn_h0=False, learn_c0=False,
                 activation_h=nn.Tanh,
                 activation_o=nn.Sigmoid, activation_f=nn.Tanh, 
                 activation_i=nn.Sigmoid, activation_j=nn.Sigmoid, 
                 rnn_mode=True):
        super().__init__()
        # it is fine to hard code these 
        self.activation_h = activation_h()
        self.activation_o = activation_o()
        self.activation_f = activation_f()
        self.activation_i = activation_i()
        self.activation_j = activation_j()
        
        # parameters of the (recurrent) hidden layer
        self.W_o = nn.Parameter(torch.randn(dims, hidden_dims) * .1)
        self.b_o = nn.Parameter(torch.zeros(1, hidden_dims))
        self.W_f = nn.Parameter(torch.randn(dims, hidden_dims) * .1)
        self.b_f = nn.Parameter(torch.zeros(1, hidden_dims))
        self.W_i = nn.Parameter(torch.randn(dims, hidden_dims) * .1)
        self.b_i = nn.Parameter(torch.zeros(1, hidden_dims))
        self.W_j = nn.Parameter(torch.randn(dims, hidden_dims) * .1)
        self.b_j = nn.Parameter(torch.zeros(1, hidden_dims))
        
        
        self.U_o = nn.Parameter(
            torch.randn(hidden_dims, hidden_dims) * .1
        )
        self.U_f = nn.Parameter(
            torch.randn(hidden_dims, hidden_dims) * .1
        )
        self.U_i = nn.Parameter(
            torch.randn(hidden_dims, hidden_dims) * .1
        )
        self.U_j = nn.Parameter(
            torch.randn(hidden_dims, hidden_dims) * .1
        )
        
        if not rnn_mode:
            self.U_o.zero_(); self.U_f.zero_(); self.U_i.zero_(); self.U_j.zero_()
            self.U_o.requires_grad = False
            self.U_f.requires_grad = False
            self.U_i.requires_grad = False
            self.U_j.requires_grad = False

        # initial hidden state
        self.h_0 = nn.Parameter(
            torch.zeros(1, hidden_dims),
            requires_grad=learn_h0 # only train this if enabled
        )        
        
        # initial cell state
        self.c_0 = nn.Parameter(
            torch.zeros(1, hidden_dims),
            requires_grad=learn_c0 # only train this if enabled
        )
        
        # output layer (fully connected)
        self.W_y = nn.Parameter(torch.randn(hidden_dims, dims) * .1)
        self.b_y = nn.Parameter(torch.zeros(1, dims))
                
    def step(self, x_t, h, c):
        #  forward pass for a single time step
        # hint: a more clever implementation could combine all these and select the different parts later
        j = self.activation_j(
            torch.matmul(x_t, self.W_j) + torch.matmul(h, self.U_j) + self.b_j
        )
        i = self.activation_i(
            torch.matmul(x_t, self.W_i) + torch.matmul(h, self.U_i) + self.b_i
        )
        f = self.activation_f(
            torch.matmul(x_t, self.W_f) + torch.matmul(h, self.U_f) + self.b_f
        )        
        o = self.activation_o(
            torch.matmul(x_t, self.W_o) + torch.matmul(h, self.U_o) + self.b_o
        )
        
        
        c = f * c + i * j
        
        h = o * self.activation_h(c)

        return h, c # returning new hidden and cell state

    def iterate_series(self, x, h, c):
        # apply rnn to each time step and give an output (many-to-many task)
        batch_size, n_steps, dimensions = x.shape
        
        # can use cell states list here but only the last cell is required
        hidden_states = []
        # iterate over time axis (1)
        for t in range(n_steps):
            # give previous hidden state and input from the current time step
            h, c = self.step(x[:, t], h, c)
            hidden_states.append(h)
        hidden_states = torch.stack(hidden_states, 1)
        
        # fully connected output
        y_hat = hidden_states.reshape(batch_size * n_steps, -1) # flatten steps and batch size (bs * )
        y_hat = torch.matmul(y_hat, self.W_y) + self.b_y
        y_hat = y_hat.reshape(batch_size, n_steps, -1) # regains structure
        return y_hat, hidden_states[:, -1], c
    
    def forward(self, x, h, c):
        # x: b, t, d
        batch_size = x.shape[0] 
        if h is None:
            h = self.h_0.repeat_interleave(batch_size, 0)
        if c is None:
            c = self.c_0.repeat_interleave(batch_size, 0)
        y_hat, h, c = self.iterate_series(x, h, c)
        return y_hat, h, c
    
    def auto_regression(self, x_0, h, c, steps):
        # one-to-many task (steps = \delta')
        x_prev = x_0
        y_hat = []
        # iterate over time axis (1)
        for t in range(steps):
            # give previous hidden state and input from the current time step
            h, c = self.step(x_prev, h, c)
            # here we need to apply the output layer on each step individually
            x_prev = torch.matmul(h, self.W_y) + self.b_y
            
            y_hat.append(x_prev)
        y_hat = torch.stack(y_hat, 1)
        return y_hat, h, c
    
    def many_to_one(self, x, h, c):
        # not required
        # returns the last prediction and the hidden state
        y_hat, h, c = self(x, h, c)
        return y_hat[:, -1], h, c # only return the last prediction
        
    def many_to_many_async(self, x, h, c, steps):
        # not required
        # combines many-to-one and one-to-many
        x_0, h, c = self.many_to_one(x, h, c)
        return self.auto_regression(x_0, h, c, steps)

In [5]:
# use the hidden state (True) by activating learning for U_h
rnn_mode = True
# learn a better initial hidden state representation (True) than zero init
learn_h0 = True

# number of weight update iterations
n_iterations = 25 #250

# number of neurons m in the hidden layer
num_neurons = 256
batch_size  = 1024 #should be 1024
lr = 0.01

device = "cpu"

In [6]:
rnn = LSTM(
    dims, num_neurons, 
    learn_h0=learn_h0, 
    rnn_mode=rnn_mode
)
rnn.to(device)

# standard optimizer SGD or AdaGrad would also work
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [7]:
rnn.train()
iteration = 0
# how often we plot the prediction
log_iter = 10
iter_loss = []

# train for a set number of iterations
for iteration in range(n_iterations):

    # generates a long time series / normally loaded from dataset (e.g. stocks, weather)
    x, y = generateData(
        dims, signal_length, predict_ahead, signal_repeats, batch_size, noise_strength
    )

    x, y = x.to(device), y.to(device)
    h = c = None
    # reset gradients
    optimizer.zero_grad()
    
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

#    t0 = time.time()
    start.record()

    # get predictions (forward pass)
    y_hat, h, c = rnn(x, h, c)
           
    # calculate mean squared error
    loss = torch.mean((y_hat - y)**2)        
    # backprop
    loss.backward()

    end.record()
#    torch.cuda.current_stream().synchronize()
#    t1 = time.time()
#    print(t1-t0)

    # Waits for everything to finish running
    torch.cuda.synchronize()

    print(start.elapsed_time(end))

    print(rnn.W_o.grad)
    # finally we are adapting the weights with the saved gradient information
    optimizer.step()


    # log loss
#    iter_loss.append(loss.item())
    
    # plot model predictions during training
#    if iteration % log_iter == 0:
#        replot(x[0].cpu(), y_hat[0].detach().cpu(), 
#               y[0].cpu(), iteration + 1, iter_loss)

5181.453125
tensor([[-1.4667e-05, -2.0047e-04,  1.7706e-04,  ...,  3.4106e-04,
          1.2953e-03,  4.7017e-04],
        [-1.4672e-05, -2.0049e-04,  1.7697e-04,  ...,  3.4101e-04,
          1.2952e-03,  4.7008e-04],
        [-1.4652e-05, -2.0062e-04,  1.7708e-04,  ...,  3.4107e-04,
          1.2954e-03,  4.7034e-04],
        ...,
        [-1.3276e-05, -2.0094e-04,  1.7662e-04,  ...,  3.4115e-04,
          1.2981e-03,  4.7479e-04],
        [-1.3210e-05, -2.0087e-04,  1.7664e-04,  ...,  3.4117e-04,
          1.2983e-03,  4.7485e-04],
        [-1.3163e-05, -2.0078e-04,  1.7655e-04,  ...,  3.4113e-04,
          1.2982e-03,  4.7487e-04]])
22934.837890625
tensor([[-3.2196e-05, -9.4095e-06, -1.1622e-03,  ...,  2.6251e-04,
          7.7918e-04, -1.3477e-04],
        [-3.2205e-05, -9.4168e-06, -1.1620e-03,  ...,  2.6253e-04,
          7.7915e-04, -1.3480e-04],
        [-3.2179e-05, -9.3867e-06, -1.1617e-03,  ...,  2.6250e-04,
          7.7927e-04, -1.3475e-04],
        ...,
        [-3.1372e-

4853.216796875
tensor([[ 2.2407e-06, -1.1990e-06, -6.8054e-06,  ..., -9.3038e-05,
         -4.9386e-06, -1.9673e-07],
        [ 2.2459e-06, -1.1964e-06, -6.8075e-06,  ..., -9.3159e-05,
         -5.0371e-06, -1.9869e-07],
        [ 2.2528e-06, -1.1993e-06, -6.7948e-06,  ..., -9.3061e-05,
         -5.0039e-06, -1.9734e-07],
        ...,
        [ 2.4567e-06, -1.1497e-06, -6.5046e-06,  ..., -9.2779e-05,
         -3.7946e-06, -2.9364e-07],
        [ 2.4543e-06, -1.1492e-06, -6.4976e-06,  ..., -9.2733e-05,
         -3.7322e-06, -2.9762e-07],
        [ 2.4573e-06, -1.1494e-06, -6.4982e-06,  ..., -9.2718e-05,
         -3.7371e-06, -2.9918e-07]])
4457.904296875
tensor([[ 2.4432e-06,  7.2040e-08, -1.3864e-05,  ..., -3.6210e-05,
          3.3344e-05, -6.3955e-07],
        [ 2.4482e-06,  7.2767e-08, -1.3871e-05,  ..., -3.6249e-05,
          3.3299e-05, -6.4118e-07],
        [ 2.4547e-06,  7.1656e-08, -1.3849e-05,  ..., -3.6248e-05,
          3.3255e-05, -6.3973e-07],
        ...,
        [ 2.6813